Import necessary library. Make sure you are running your google colab using GPU

In [1]:
import os
import json
import argparse
import time
import numpy as np
import pandas as pd
import datetime
import tensorflow as tf
import time
from pickle import TRUE
from tensorflow.keras import layers
from keras.models import load_model
from sklearn import utils
from tqdm import tqdm as tqdm
from sklearn import metrics 
import warnings
warnings.filterwarnings('ignore')

print("GPU Condition", tf.test.is_gpu_available()) # True/False

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU Condition True


# **Next Activity Prediction Task**

# Preprocessing Part
Upload necessary dataset to your directory. We are using Helpdesk dataset as example.
*   **Make sure the dataset has columns name Case ID, Activity, Complete Timestamp, and Resource (If not, please rename it manually)**

You can download our sample dataset here (https://drive.google.com/drive/folders/1x2en9pUz1ak8IO_6ocE-hpkGylaz1da3?usp=sharing)

In [ ]:
#change to your prefered event log
df = pd.read_csv("Helpdesk.csv")

In [ ]:
df

,Case ID,Activity,Resource,Complete Timestamp,Variant,Variant index,Variant.1,seriousness,customer,product,responsible_section,seriousness_2,service_level,service_type,support_section,workgroup
0,Case 1,Assign seriousness,Value 1,2012/10/09 14:50:17.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
1,Case 1,Take in charge ticket,Value 1,2012/10/09 14:51:01.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
2,Case 1,Take in charge ticket,Value 2,2012/10/12 15:02:56.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
3,Case 1,Resolve ticket,Value 1,2012/10/25 11:54:26.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
4,Case 1,Closed,Value 3,2012/11/09 12:54:39.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21343,Case 4579,Closed,Value 5,2010/09/02 10:11:00.000,Variant 1,1,Variant 1,Value 1,Value 71,Value 3,Value 1,Value 1,Value 3,Value 1,Value 1,Value 1
21344,Case 4580,Take in charge ticket,Value 6,2012/01/03 09:33:43.000,Variant 18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21345,Case 4580,Wait,Value 6,2012/01/10 15:30:11.000,Variant 18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21346,Case 4580,Resolve ticket,Value 6,2012/01/10 17:07:40.000,Variant 18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1


In [ ]:
df = df[['Case ID', 'Activity', 'Complete Timestamp', 'Resource']]
df.columns = ["Case", "Activity", "Timestamp", 'Resource']

In [ ]:
df

,Case,Activity,Timestamp,Resource
0,Case 1,Assign seriousness,2012/10/09 14:50:17.000,Value 1
1,Case 1,Take in charge ticket,2012/10/09 14:51:01.000,Value 1
2,Case 1,Take in charge ticket,2012/10/12 15:02:56.000,Value 2
3,Case 1,Resolve ticket,2012/10/25 11:54:26.000,Value 1
4,Case 1,Closed,2012/11/09 12:54:39.000,Value 3
...,...,...,...,...
21343,Case 4579,Closed,2010/09/02 10:11:00.000,Value 5
21344,Case 4580,Take in charge ticket,2012/01/03 09:33:43.000,Value 6
21345,Case 4580,Wait,2012/01/10 15:30:11.000,Value 6
21346,Case 4580,Resolve ticket,2012/01/10 17:07:40.000,Value 6


In [ ]:
df["Activity"] = df["Activity"].str.lower()
df["Activity"] = df["Activity"].str.replace(" ", "-")

df["Timestamp"] = df["Timestamp"].str.replace("/", "-")
df["Timestamp"]= pd.to_datetime(df["Timestamp"], dayfirst=True).map(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
df.sort_values(by = ["Timestamp"], inplace = True)

NameError: ignored

In [ ]:
df

,Case,Activity,Timestamp,Resource
0,Case 1,assign-seriousness,2012-10-09 14:50:17,Value 1
1,Case 1,take-in-charge-ticket,2012-10-09 14:51:01,Value 1
2,Case 1,take-in-charge-ticket,2012-10-12 15:02:56,Value 2
3,Case 1,resolve-ticket,2012-10-25 11:54:26,Value 1
4,Case 1,closed,2012-11-09 12:54:39,Value 3
...,...,...,...,...
21343,Case 4579,closed,2010-09-02 10:11:00,Value 5
21344,Case 4580,take-in-charge-ticket,2012-01-03 09:33:43,Value 6
21345,Case 4580,wait,2012-01-10 15:30:11,Value 6
21346,Case 4580,resolve-ticket,2012-01-10 17:07:40,Value 6


In [ ]:
keys = ["[PAD]", "[UNK]"]
activities = list(df["Activity"].unique())
resources = list(df["Resource"].unique())
resources = list(map(str, resources))

keys.extend(activities)
val = range(len(keys))

coded_activity = dict({"x_word_dict":dict(zip(keys, val))})
code_activity_normal = dict({"y_word_dict": dict(zip(activities, range(len(activities))))})
code_resource_normal = dict({"x2_word_dict": dict(zip(resources, range(len(resources))))})

coded_activity.update(code_activity_normal)
coded_activity.update(code_resource_normal)

coded_json = json.dumps(coded_activity)
with open(f"metadata.json", "w") as metadata_file:
    metadata_file.write(coded_json)

In [ ]:
train_test_ratio = int(abs(df["Case"].nunique()*0.80))
train_list = df["Case"].unique()[:train_test_ratio]
test_list = df["Case"].unique()[train_test_ratio:]

In [ ]:
case_id = "Case"
case_name = "Activity"
case_resource = "Resource"

processed_df = pd.DataFrame(columns = ["case_id", "resource", "prefix", "k", "next_act"])
idx = 0
unique_cases = df[case_id].unique()

for _, case in enumerate(tqdm(unique_cases)):
    act = df[df[case_id] == case][case_name].to_list()
    resource = df[df[case_id] == case][case_resource].to_list()
    for i in range(len(act) - 1):
        prefix = np.where(i == 0, act[0], " ".join(act[:i+1]))
        next_act = act[i+1]
        resource_act = resource[i]

        processed_df.at[idx, "case_id"] = case
        processed_df.at[idx, "resource"] = resource_act
        processed_df.at[idx, "prefix"] = prefix
        processed_df.at[idx, "k"] = i
        processed_df.at[idx, "next_act"] = next_act
        idx = idx + 1

100%|██████████| 4580/4580 [00:36<00:00, 126.68it/s]


In [ ]:
train_remaining_time = processed_df[processed_df["case_id"].isin(train_list)]
test_remaining_time = processed_df[processed_df["case_id"].isin(test_list)]
train_remaining_time.to_csv(f"train.csv", index = False)
test_remaining_time.to_csv(f"test.csv", index = False)

# Proposal Architecture (Transformer)

In [15]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_a = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm_b = layers.LayerNormalization(epsilon=1e-6)
        self.dropout_a = layers.Dropout(rate)
        self.dropout_b = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout_a(attn_output, training=training)
        out_a = self.layernorm_a(inputs + attn_output)
        ffn_output = self.ffn(out_a)
        ffn_output = self.dropout_b(ffn_output, training=training)
        return self.layernorm_b(out_a + ffn_output)

# Next Activity Model

In [16]:
def get_next_activity_model(max_case_length, vocab_size, output_dim, 
    embed_dim = 36, num_heads = 4, ff_dim = 64):
    inputs = layers.Input(shape=(max_case_length,))
    x = TokenAndPositionEmbedding(max_case_length, vocab_size, embed_dim)(inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(output_dim, activation="linear")(x)
    transformer = tf.keras.Model(inputs=inputs, outputs=outputs,
        name = "next_activity_transformer")
    print("model", transformer.summary())
    return transformer

# Next Activity Data Loader

In [17]:
def prepare_data_next_activity(df, 
        x_word_dict, y_word_dict, 
        max_case_length, shuffle=True):
        
        x = df["prefix"].values
        y = df["next_act"].values
        if shuffle:
            x, y = utils.shuffle(x, y)

        token_x = list()
        for _x in x:
            token_x.append([x_word_dict[s] for s in _x.split()])
        # token_x = np.array(token_x, dtype = np.float32)

        token_y = list()
        for _y in y:
            token_y.append(y_word_dict[_y])
        # token_y = np.array(token_y, dtype = np.float32)

        token_x = tf.keras.preprocessing.sequence.pad_sequences(
            token_x, maxlen=max_case_length)

        token_x = np.array(token_x, dtype=np.float32)
        token_y = np.array(token_y, dtype=np.float32)

        return token_x, token_y

def get_max_case_length(train_x):
        train_token_x = list()
        for _x in train_x:
            train_token_x.append(len(_x.split()))
        return max(train_token_x)

def load_data():
        train_df = pd.read_csv(f"train.csv")
        test_df = pd.read_csv(f"test.csv")

        with open(f"metadata.json", "r") as json_file:
            metadata = json.load(json_file)

        x_word_dict = metadata["x_word_dict"]
        y_word_dict = metadata["y_word_dict"]
        max_case_length = get_max_case_length(train_df["prefix"].values)
        vocab_size = len(x_word_dict) 
        total_classes = len(y_word_dict)

        return (train_df, test_df, 
            x_word_dict, y_word_dict, 
            max_case_length, vocab_size, 
            total_classes)

# Training

In [18]:
(train_df, test_df, x_word_dict, y_word_dict, max_case_length, vocab_size, num_output) = load_data()
train_token_x, train_token_y = prepare_data_next_activity(train_df, x_word_dict, y_word_dict, max_case_length)

In [19]:
transformer_model = get_next_activity_model(
        max_case_length=max_case_length, 
        vocab_size=vocab_size,
        output_dim=num_output)

Model: "next_activity_transformer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 14)]              0         
                                                                 
 token_and_position_embeddin  (None, 14, 36)           1080      
 g_2 (TokenAndPositionEmbedd                                     
 ing)                                                            
                                                                 
 transformer_block_2 (Transf  (None, 14, 36)           26056     
 ormerBlock)                                                     
                                                                 
 global_average_pooling1d_2   (None, 36)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_10 (Dropout)        (None, 36)  

In [20]:
transformer_model.compile(optimizer=tf.keras.optimizers.Adam(0.001), 
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

In [21]:
transformer_model.fit(train_token_x, train_token_y, 
                      epochs=50, batch_size=256, 
                      shuffle=True, verbose=2)

Epoch 1/50
53/53 - 2s - loss: 1.6957 - sparse_categorical_accuracy: 0.3502 - 2s/epoch - 31ms/step
Epoch 2/50
53/53 - 0s - loss: 0.9327 - sparse_categorical_accuracy: 0.7423 - 338ms/epoch - 6ms/step
Epoch 3/50
53/53 - 0s - loss: 0.7505 - sparse_categorical_accuracy: 0.7757 - 319ms/epoch - 6ms/step
Epoch 4/50
53/53 - 0s - loss: 0.7089 - sparse_categorical_accuracy: 0.7779 - 322ms/epoch - 6ms/step
Epoch 5/50
53/53 - 0s - loss: 0.6890 - sparse_categorical_accuracy: 0.7855 - 304ms/epoch - 6ms/step
Epoch 6/50
53/53 - 0s - loss: 0.6703 - sparse_categorical_accuracy: 0.7902 - 317ms/epoch - 6ms/step
Epoch 7/50
53/53 - 0s - loss: 0.6627 - sparse_categorical_accuracy: 0.7902 - 315ms/epoch - 6ms/step
Epoch 8/50
53/53 - 0s - loss: 0.6594 - sparse_categorical_accuracy: 0.7909 - 310ms/epoch - 6ms/step
Epoch 9/50
53/53 - 0s - loss: 0.6511 - sparse_categorical_accuracy: 0.7912 - 301ms/epoch - 6ms/step
Epoch 10/50
53/53 - 0s - loss: 0.6472 - sparse_categorical_accuracy: 0.7920 - 308ms/epoch - 6ms/step
E

# Evaluate Model

In [22]:
k, accuracies,fscores, precisions, recalls = [],[],[],[],[]
for i in range(max_case_length):
    test_data_subset = test_df[test_df["k"]==i]
    if len(test_data_subset) > 0:
        test_token_x, test_token_y = prepare_data_next_activity(test_data_subset, 
            x_word_dict, y_word_dict, max_case_length)   
        y_pred = np.argmax(transformer_model.predict(test_token_x), axis=1)
        accuracy = metrics.accuracy_score(test_token_y, y_pred)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            test_token_y, y_pred, average="weighted")
        k.append(i)
        accuracies.append(accuracy)
        fscores.append(fscore)
        precisions.append(precision)
        recalls.append(recall)

k.append(i + 1)
accuracies.append(np.mean(accuracy))
fscores.append(np.mean(fscores))
precisions.append(np.mean(precisions))
recalls.append(np.mean(recalls))
print('Average accuracy across all prefixes:', np.mean(accuracies))
print('Average f-score across all prefixes:', np.mean(fscores))
print('Average precision across all prefixes:', np.mean(precisions))
print('Average recall across all prefixes:', np.mean(recalls))    

Average accuracy across all prefixes: 0.8508993241296211
Average f-score across all prefixes: 0.8093510907817407
Average precision across all prefixes: 0.8138327232734779
Average recall across all prefixes: 0.8384742678070896
